# Data Cleaning and Processing

## Introduction

This notebook goes through a necessary step of any data science project - data cleaning. Data cleaning is very important. Keep in mind, "garbage in, garbage out". Feeding dirty data into a model will give us results that are meaningless.

## Getting Data from Data Base

In [62]:
import re
import string
import sqlite3

import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk import RegexpTokenizer, PorterStemmer, WordNetLemmatizer, FreqDist

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns; sns.set()

from utils import * # import all existing functions from file utils.py

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
conn = sqlite3.connect('../data/business_places.db')
cursor = conn.cursor()

In [64]:
data = pd.DataFrame(cursor.execute("SELECT * FROM corpus").fetchall())
data.columns = [x[0] for x in cursor.description]
data = data.drop(columns = ['index'])
data.head()

,name,location,rating,date,business_id,text
0,Alan H.,"Tampa, FL",Neutral,3/2/2020,KvLrI20Abz6F9WzAIHmyWQ,With Cafe R and the hotel having a deal for us...
1,Shraddha R.,"New York, NY",Negative,5/4/2019,KvLrI20Abz6F9WzAIHmyWQ,Went here because we had a food coupon we didn...
2,Michelle L.,"Washington Heights, Manhattan, NY",Negative,7/21/2020,KvLrI20Abz6F9WzAIHmyWQ,As you guys are aware majority of places are c...
3,Paul L.,"North Babylon, NY",Negative,1/26/2020,KvLrI20Abz6F9WzAIHmyWQ,Went here fo continental breakfast and this pl...
4,Andrew G.,"Manhattan, NY",Positive,11/27/2019,KvLrI20Abz6F9WzAIHmyWQ,Great cafe located in a busy section of Chelse...


In [65]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37907 entries, 0 to 37906
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         37907 non-null  object
 1   location     37907 non-null  object
 2   rating       37907 non-null  object
 3   date         37907 non-null  object
 4   business_id  37907 non-null  object
 5   text         37907 non-null  object
dtypes: object(6)
memory usage: 1.7+ MB


## Cleaning The Data

When dealing with numerical data, data cleaning often involves removing null values and duplicate data, dealing with outliers, etc. With text data, there are some common data cleaning techniques, which are also known as text pre-processing techniques.

With text data, this cleaning process can go on forever. There's always an exception to every cleaning step. So, we're going to follow the MVP (minimum viable product) approach - start simple and iterate. Here are a bunch of things you can do to clean your data. We're going to execute just the common cleaning steps here and the rest can be done at a later point to improve our results.



#### Common data cleaning steps on all text:

* Make text all lower case
* Remove punctuation
* Remove numerical values
* Remove common non-sensical text (/n)
* Tokenize text
* Remove stop words

#### More data cleaning steps after tokenization:

* Stemming / lemmatization
* Parts of speech tagging
* Create bi-grams or tri-grams
* Deal with typos
* And more...

## Setting corpus First


In [66]:
corpus = data.loc[:, ['text','location','rating']]

## Lets clean text of each review with function cleaning_text from utils

In [67]:
corpus.loc[:,'clean_text'] = corpus.loc[:,'text'].apply(cleaning_text)

# Words with 1-2 letters that can be usless we can use filter_review function to clean this words
corpus.loc[:,'clean_text'] = corpus.loc[:, 'clean_text'].apply(filter_review)

##  Tokenization 
Tokenization is the process of splitting documents into units of observations. We usually represent the tokens as __n-gram__, where n represent the consecutive words occuring in a document.

In [68]:
corpus.loc[:,'token_text'] = corpus.loc[:, 'clean_text'].apply(tokenize_text)

## Stemming / Lemmatization

##### Stemming 
Stemming allows us to remove different variations of the same word. For example, collect, collection and collecting will all be reduced to the same single word collect.
- Stemming is the process of reducing inflection in words to their root forms, such as mapping a group of words to the same stem even if the stem itself is not a valid word in the Language.
- Stems are created by removing the suffixes or prefixes used with a word.

#### Lemmatization
The only difference between lemmatization and stemming is that lemmatization returns real words. For example, instead of returning "movi" like Porter stemmer would, "movie" will be returned by the lemmatizer.

- Unlike Stemming, Lemmatization reduces the inflected words properly ensuring that the root word belongs to the language. 

- In Lemmatization, the root word is called Lemma. 

- A lemma (plural lemmas or lemmata) is the canonical form, dictionary form, or citation form of a set of words.

In [69]:
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

corpus['lem'] = corpus['token_text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
corpus['stem'] = corpus['token_text'].apply(lambda x: [ps.stem(word) for word in x])

## Create syntactic features

Using NLTK to create context-free grammar and part-of-speech (POS) tags

In [70]:
corpus['pos']  = corpus['token_text'].apply(lambda x: [pos_tag[1] for pos_tag in pos_tag(x)])    # bag of tags

# calculate percentage of nouns in each review
corpus['noun'] = corpus['pos'].apply(lambda x: sum(1 for pos in x if pos.startswith('NN')) / len(x) if len(x) > 0 else 0) 

# calculate percentage of adjectives in each review
corpus['adj']  = corpus['pos'].apply(lambda x: sum(1 for pos in x if pos.startswith('JJ')) / len(x) if len(x) > 0 else 0)

# calculate percentage of adverbs in each review
corpus['adv'] = corpus['pos'].apply(lambda x: sum(1 for pos in x if pos.startswith('RB')) / len(x) if len(x) > 0 else 0)

# calculate percentage of verbs in each review
corpus['verb'] = corpus['pos'].apply(lambda x: sum(1 for pos in x if pos.startswith('VB')) / len(x) if len(x) > 0 else 0)

## Create Structual features

In [71]:
corpus['characters'] = corpus['text'].apply(lambda x: len(x))            # number of characters
corpus['tokens'] = corpus['token_text'].apply(lambda x: len(x))          # number of tokens
corpus['words'] = corpus['text'].apply(lambda x: len(x.split(' ')))      # number of words
corpus['sentences'] = corpus['text'].apply(lambda x: len(x.split('. '))) # number of sentences
corpus['avg_word_len'] = corpus['characters'] / corpus['words']          # average word length
corpus['avg_sent_len'] = corpus['words'] / corpus['sentences']           # average sentence length

## Location specify home place of the person who left review. We can turn this column into categorical feature

In [72]:
corpus.loc[:, 'location'] = corpus.loc[:, 'location'].map(lambda x : 1 if 'NY' in x else 0)

In [73]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37907 entries, 0 to 37906
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   text          37907 non-null  object 
 1   location      37907 non-null  int64  
 2   rating        37907 non-null  object 
 3   clean_text    37907 non-null  object 
 4   token_text    37907 non-null  object 
 5   lem           37907 non-null  object 
 6   stem          37907 non-null  object 
 7   pos           37907 non-null  object 
 8   noun          37907 non-null  float64
 9   adj           37907 non-null  float64
 10  adv           37907 non-null  float64
 11  verb          37907 non-null  float64
 12  characters    37907 non-null  int64  
 13  tokens        37907 non-null  int64  
 14  words         37907 non-null  int64  
 15  sentences     37907 non-null  int64  
 16  avg_word_len  37907 non-null  float64
 17  avg_sent_len  37907 non-null  float64
dtypes: float64(6), int64(5), o

### Saving cleaned corpus to DB and CSV

In [74]:
corpus.to_csv('../data/corpus_processed.csv')

In [75]:
corpus = corpus.drop(columns = 'pos')

corpus['lem'] = corpus['lem'].apply(lambda x : ' '.join(x))
corpus['stem'] = corpus['stem'].apply(lambda x : ' '.join(x))
corpus['token_text'] = corpus['token_text'].apply(lambda x : ' '.join(x))

In [76]:
cursor.execute('DROP TABLE IF EXISTS {};'.format('corpus_processed'))

In [77]:
corpus.to_sql('corpus_processed', con= conn)

In [78]:
cursor.close()
conn.close()